# itch_cross_response_data

#### Juan Camilo Henao Londono - 07.02.2019
#### AG Guhr - Universitaet Duisburg-Essen

$$ r_{i} (t,\tau) = \log m_{i}(t + \tau) - \log m_{i}(t) = \log \frac{m_{i}(t + \tau)}{m_{i}(t)} $$
$$ R_{ij}^{(exc. 0)}(\tau) = \left \langle r_i(t,\tau) \epsilon_{j} (t) \right \rangle_t^{(exc. 0)} $$

In the following [link]() can be seen the plot of the function's results

In [1]:
# Import modules

import numpy as np
import os

import gzip
import pickle

import itch_data_tools

__tau__ = 1000

In [2]:
def itch_cross_response_data(ticker_i, ticker_j, year, month, day, tau_val,
                             t_step):
    """
    Obtain the cross response function using the midpoint log returns of
    ticker i and trade signs of ticker j during different time lags. The data
    is adjusted to use only the values each t_step ms
        :param ticker_i: string of the abbreviation of the midpoint stock to
         be analized (i.e. 'AAPL')
        :param ticker_j: string of the abbreviation of the trade sign stock to
         be analized (i.e. 'AAPL')
        :param year: string of the year to be analized (i.e '2016')
        :param month: string of the month to be analized (i.e '07')
        :param day: string of the day to be analized (i.e '07')
        :param tau_val: maximum time lag to be analyzed
        :param t_step: time step in the data in ms
    """
    if (ticker_i == ticker_j):

        return None

    else:

        function_name = itch_cross_response_data.__name__
        itch_data_tools.itch_function_header_print_data(function_name,
                                                        ticker_i, ticker_j,
                                                        year, month, day,
                                                        str(t_step))

        # Load data
        midpoint_i = pickle.load(open(''.join((
                '../itch_data_{1}/itch_midpoint_data_1ms/itch_midpoint_data'
                + '_midpoint_{1}{2}{3}_{0}_1ms.pickle').split())
                .format(ticker_i, year, month, day), 'rb'))
        trade_sign_j = pickle.load(open("".join((
                '../itch_data_{1}/itch_trade_signs_data_1ms/itch_trade_signs'
                + '_data_{1}{2}{3}_{0}_1ms.pickle').split())
                .format(ticker_j, year, month, day), 'rb'))
        time = pickle.load(open(''.join((
                '../itch_data_{}/itch_midpoint_data_1ms/itch_midpoint_data'
                + '_time_1ms.pickle').split())
                .format(year), 'rb'))

        # Setting variables to work with t_step ms accuracy

        # Array of the average of each tau. 10^3 s used by Wang
        cross_response_tau = np.zeros(__tau__)

        # Using values each second
        midpoint_i_sec = midpoint_i[::t_step]
        # Changing time from 1 ms to t_step ms
        time_t_step = time[::t_step]

        # reshape and average data of trade signs
        (trade_sign_j_sec_avg,
         trade_sign_j_sec_nr) = itch_data_tools.itch_trade_sign_reshape(
                                                    trade_sign_j, time_t_step)

        # Calculating the midpoint log return and the cross response function

        # Depending on the ta
        for tau_idx, tau_v in enumerate(range(1, tau_val + 1,
                                        int(tau_val * 1E-3))):

            # Obtain the midpoint log return. Displace the numerator tau
            # values to the right and compute the return, and append the
            # remaining values of tau with zeros
            log_return_i_sec = np.append(np.log(
                midpoint_i_sec[tau_v:]/midpoint_i_sec[:-tau_v]),
                np.zeros(tau_v))

            cross_response_tau[tau_idx] = np.mean(
                log_return_i_sec[trade_sign_j_sec_nr != 0] *
                trade_sign_j_sec_avg[trade_sign_j_sec_nr != 0])

        # Saving data

        itch_data_tools.itch_save_data(function_name, cross_response_tau,
                                       ticker_i, ticker_j, year, month, day,
                                       str(t_step))

        return None

In [3]:
# Test

ticker_i = 'AAPL'
ticker_j = 'MSFT'
year = '2016'
month = '03'
days = ['07', '08', '09', '10', '11']
tau_val = 1000
t_step = 1000

for day in days:
    itch_cross_response_data(ticker_i, ticker_j, year, month, day, tau_val, t_step)

ITCH data
itch_cross_response_data
Processing data for the stock i AAPL and stock j MSFT the 2016.03.07
Time step: 1000ms


FileNotFoundError: [Errno 2] No such file or directory: '../itch_data_2016/itch_midpoint_data_1ms/itch_midpoint_data_midpoint_20160307_AAPL_1ms.pickle'